### Labic Images Segmentation - Main Notebook

#### Importando módulos

In [1]:
import labic_init
labic_init.labic_init()

Segmentation Models: using `keras` framework.
Inicializando...
Segmentação com modelo Linknet_resnet34 inicializada com sucesso.


In [2]:
from labic_images_segmentation import Dataset, DataAugmentation, SegmentationModel, SaveReport, PredictImages, DiceCoef

In [3]:
import tensorflow as tf
import json

#### Diretórios e parâmetros

In [4]:
with open("config.json", "r") as params:
    config = json.load(params)

In [ ]:
""" if config["callback"] == "early_stopping":
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=10) """

#### Treinamento

##### Carregando dataset

In [5]:
imgs = Dataset(folder=config["dataset_folder"], norm_imgs_folder=config["norm_imgs_folder"], gt_folder=config["gt_folder"], \
                ORIGINAL_SIZE=config["ORIGINAL_SIZE"], NEW_SIZE=config["NEW_SIZE"])


Imagens carregadas com sucesso.


##### Dividindo dataset 

In [6]:
imgs.split_dataset()


Dataset subdividido.
20.0% dos dados para validação.
80.0% dos dados para treino.


##### Data Augmentation

In [7]:
data_aug = DataAugmentation(X_train=imgs.X_train, Y_train=imgs.Y_train, X_val=imgs.X_val, Y_val=imgs.Y_val, use_batch_size=config["batch_size"])

##### Treinamento da rede

In [8]:
segment = SegmentationModel(N=imgs.X_train.shape[-1], segmentation_model=config["model"], backbone_name=config["backbone_name"],\
                             trainDS=data_aug.trainDS, valDS=data_aug.valDS, epochs=config["epochs"])

Epoch 1/3
233/233 [==============================] - 28s 74ms/step - loss: 0.3230 - iou_score: 0.7312 - val_loss: 0.3509 - val_iou_score: 0.7055
Epoch 2/3
233/233 [==============================] - 16s 66ms/step - loss: 0.0818 - iou_score: 0.9294 - val_loss: 0.1984 - val_iou_score: 0.8285
Epoch 3/3
233/233 [==============================] - 16s 66ms/step - loss: 0.0498 - iou_score: 0.9564 - val_loss: 0.0370 - val_iou_score: 0.9668


##### Salvando modelo e history

In [9]:
save_report = SaveReport(model=segment.model, history=segment.history, folder_name=config["output_folder"], n_fold=config["n_fold"],\
                         use_batch_size=config["batch_size"], epochs=config["epochs"], exec_folder_name=config["exec_folder_name"])

Diretório  ./TM40_46Prod/outputs/Exec_2023-05-05-12-48-02.871869  criado.
Diretório  ./TM40_46Prod/outputs/Exec_2023-05-05-12-48-02.871869/fold_0  criado.

Modelo salvo.
Nome: model_4_3_exec_2023-05-05-12-48-02.871869_fold_0
Salvo em: ./TM40_46Prod/outputs/Exec_2023-05-05-12-48-02.871869
Arquivos history_0.csv/txt salvos com sucesso.


#### Predição 

##### Carregando imagens de teste

In [10]:
test_imgs = Dataset(folder=config["output_folder"], norm_imgs_folder=config["test_imgs_folder"], gt_folder=config["gt_test_folder"],\
                     ORIGINAL_SIZE=config["ORIGINAL_SIZE"], NEW_SIZE=config["NEW_SIZE"])


Imagens carregadas com sucesso.


#### Segmentando imagens de teste

In [11]:
prediction = PredictImages(test_images=test_imgs, n_fold_folder_name=save_report.n_fold_folder_name, \
                           model_name=save_report.model_name, use_batch_size=config["batch_size"], img_shape=test_imgs.img_shape)

Diretório  ./TM40_46Prod/outputs/Exec_2023-05-05-12-48-02.871869/fold_0/outputs_prod  criado.


c:\Yana\girino_att_Yana\girino-segmentation\labic_images_segmentation.py:466: UserWarning: ./TM40_46Prod/outputs/Exec_2023-05-05-12-48-02.871869/fold_0/outputs_prod/predicao_025_4.png is a low contrast image
  io.imsave(self.n_fold_folder_name + '/outputs_prod/predicao_%s_%s.png'%(str(self.test_images.GT_imgs[i][-7:-4]), str(self.batch)),\
c:\Yana\girino_att_Yana\girino-segmentation\labic_images_segmentation.py:466: UserWarning: ./TM40_46Prod/outputs/Exec_2023-05-05-12-48-02.871869/fold_0/outputs_prod/predicao_026_4.png is a low contrast image
  io.imsave(self.n_fold_folder_name + '/outputs_prod/predicao_%s_%s.png'%(str(self.test_images.GT_imgs[i][-7:-4]), str(self.batch)),\
c:\Yana\girino_att_Yana\girino-segmentation\labic_images_segmentation.py:466: UserWarning: ./TM40_46Prod/outputs/Exec_2023-05-05-12-48-02.871869/fold_0/outputs_prod/predicao_027_4.png is a low contrast image
  io.imsave(self.n_fold_folder_name + '/outputs_prod/predicao_%s_%s.png'%(str(self.test_images.GT_imgs[i][-7


Imagens preditas com sucesso.


c:\Yana\girino_att_Yana\girino-segmentation\labic_images_segmentation.py:466: UserWarning: ./TM40_46Prod/outputs/Exec_2023-05-05-12-48-02.871869/fold_0/outputs_prod/predicao_425_4.png is a low contrast image
  io.imsave(self.n_fold_folder_name + '/outputs_prod/predicao_%s_%s.png'%(str(self.test_images.GT_imgs[i][-7:-4]), str(self.batch)),\
c:\Yana\girino_att_Yana\girino-segmentation\labic_images_segmentation.py:466: UserWarning: ./TM40_46Prod/outputs/Exec_2023-05-05-12-48-02.871869/fold_0/outputs_prod/predicao_426_4.png is a low contrast image
  io.imsave(self.n_fold_folder_name + '/outputs_prod/predicao_%s_%s.png'%(str(self.test_images.GT_imgs[i][-7:-4]), str(self.batch)),\
c:\Yana\girino_att_Yana\girino-segmentation\labic_images_segmentation.py:466: UserWarning: ./TM40_46Prod/outputs/Exec_2023-05-05-12-48-02.871869/fold_0/outputs_prod/predicao_427_4.png is a low contrast image
  io.imsave(self.n_fold_folder_name + '/outputs_prod/predicao_%s_%s.png'%(str(self.test_images.GT_imgs[i][-7

##### Salvando DSC

In [12]:
dice = DiceCoef(gt_imgs=test_imgs.Y, pred_folder=prediction.n_fold_folder_name+"/outputs_prod/*", new_size=config["NEW_SIZE"])
n_fold = config["n_fold"]
dice.save_dice(f"{save_report.n_fold_folder_name}/dice_fold_{n_fold}.txt")

Coeficiente Dice: 0.924710588915499


##### DSC final (n-folds)

In [13]:
dice.generate_csv_dice(save_report=save_report, n_all_folders = config["n_fold"], title=config["title"])

Arquivo Linknet_resnet34.csv gerado com sucesso.


##### Gráficos - Performance do Treino

In [14]:
dice.generate_graphic(segment=segment, save_report=save_report, epochs=config["epochs"], graphic_type="iou_score")

In [15]:
dice.generate_graphic(segment=segment, save_report=save_report, epochs=config["epochs"], graphic_type="loss")

##### DSC final (n-folds)

In [16]:
dice.generate_csv_dice(save_report=save_report, n_all_folders = config["n_fold"], title=config["title"])

Arquivo Linknet_resnet34.csv gerado com sucesso.


In [17]:
dice.df

,Index,Linknet_resnet34
0,0,0.924711
1,Mean,0.924711
2,Std-Dev,0.000000
3,Median,0.924711
4,Max,0.924711
5,Min,0.924711
